# Merging VCFs sample-wise

See https://github.com/pystatgen/sgkit/issues/840
    
How can we merge VCFs that contain one sample per VCF, and where some variants are not in the other file?

It's normally preferable to do this with [bcftools merge](https://samtools.github.io/bcftools/bcftools.html#merge), but this page shows a way to do it with Xarray.

In [1]:
! cat 1.vcf

##fileformat=VCFv4.0
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE1
chr1	14458	.	G	.	.	PASS	AC=0;AN=2	GT	0/0
chr1	14464	.	A	T	110	PASS	AC=1;AN=2	GT	0/1
chr1	14465	.	G	.	.	PASS	AC=0;AN=2	GT	0/0


In [2]:
! cat 2.vcf

##fileformat=VCFv4.0
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE2
chr1	14464	.	A	T	110	PASS	AC=1;AN=2	GT	0/1
chr1	14465	.	G	.	.	PASS	AC=0;AN=2	GT	0/0
chr1	14676	.	G	.	.	PASS	AC=0;AN=2	GT	0/0
chr1	14677	.	G	A	85	PASS	AC=1;AN=2	GT	0/1


In [3]:
import sgkit as sg
from sgkit.io.vcf import vcf_to_zarr
import xarray as xr

In [4]:
# Convert to Zarr and load as datasets
vcf_to_zarr("1.vcf.gz", "1.zarr")
vcf_to_zarr("2.vcf.gz", "2.zarr")

ds1 = sg.load_dataset("1.zarr")
ds2 = sg.load_dataset("2.zarr")

# load into memory
ds1.load()
ds2.load()

[W::vcf_parse] INFO 'AC' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'AN' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'AC' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'AN' is not defined in the header, assuming Type=String


<xarray.Dataset>
Dimensions:               (variants: 4, samples: 1, ploidy: 2, alleles: 4,
                           filters: 1)
Dimensions without coordinates: variants, samples, ploidy, alleles, filters
Data variables:
    call_genotype         (variants, samples, ploidy) int8 0 1 0 0 0 0 0 1
    call_genotype_mask    (variants, samples, ploidy) bool False False ... False
    call_genotype_phased  (variants, samples) bool False False False False
    sample_id             (samples) object 'SAMPLE2'
    variant_allele        (variants, alleles) object 'A' 'T' '' '' ... 'A' '' ''
    variant_contig        (variants) int8 0 0 0 0
    variant_filter        (variants, filters) bool True True True True
    variant_id            (variants) object '.' '.' '.' '.'
    variant_id_mask       (variants) bool True True True True
    variant_position      (variants) int32 14464 14465 14676 14677
    variant_quality       (variants) float32 110.0 nan nan 85.0
Attributes:
    contigs:               ['chr1']
    filters:               ['PASS']
    max_alt_alleles_seen:  1
    source:                sgkit-0.3.0
    vcf_header:            ##fileformat=VCFv4.0\n##FILTER=<ID=PASS,Descriptio...
    vcf_zarr_version:      0.1

In [5]:
# Restrict datasets to contig and pos (for the join index) and variables with a samples dimension
vars = ["variants", "variant_contig", "variant_position"]  # index vars
vars.extend([v for v in ds1.data_vars if "samples" in ds1[v].dims])  # vars with a samples dim

ds1 = ds1[vars]
ds2 = ds2[vars]

ds1_ind = ds1.set_index(variants=("variant_contig", "variant_position"))
ds1_ind.load()

ds2_ind = ds2.set_index(variants=("variant_contig", "variant_position"))
ds2_ind.load()

<xarray.Dataset>
Dimensions:               (variants: 4, samples: 1, ploidy: 2)
Coordinates:
  * variants              (variants) object MultiIndex
  * variant_contig        (variants) int8 0 0 0 0
  * variant_position      (variants) int32 14464 14465 14676 14677
Dimensions without coordinates: samples, ploidy
Data variables:
    call_genotype         (variants, samples, ploidy) int8 0 1 0 0 0 0 0 1
    call_genotype_mask    (variants, samples, ploidy) bool False False ... False
    call_genotype_phased  (variants, samples) bool False False False False
    sample_id             (samples) object 'SAMPLE2'
Attributes:
    contigs:               ['chr1']
    filters:               ['PASS']
    max_alt_alleles_seen:  1
    source:                sgkit-0.3.0
    vcf_header:            ##fileformat=VCFv4.0\n##FILTER=<ID=PASS,Descriptio...
    vcf_zarr_version:      0.1

In [6]:
# Concatenate the VCFs
ds = xr.concat([ds1_ind, ds2_ind], dim="samples", data_vars="minimal")
ds

<xarray.Dataset>
Dimensions:               (variants: 5, samples: 2, ploidy: 2)
Coordinates:
  * variants              (variants) object MultiIndex
  * variant_contig        (variants) int64 0 0 0 0 0
  * variant_position      (variants) int64 14458 14464 14465 14676 14677
Dimensions without coordinates: samples, ploidy
Data variables:
    call_genotype         (variants, samples, ploidy) float64 0.0 0.0 ... 1.0
    call_genotype_mask    (variants, samples, ploidy) float64 0.0 0.0 ... 0.0
    call_genotype_phased  (variants, samples) float64 0.0 nan 0.0 ... nan 0.0
    sample_id             (samples) object 'SAMPLE1' 'SAMPLE2'
Attributes:
    contigs:               ['chr1']
    filters:               ['PASS']
    max_alt_alleles_seen:  1
    source:                sgkit-0.3.0
    vcf_header:            ##fileformat=VCFv4.0\n##FILTER=<ID=PASS,Descriptio...
    vcf_zarr_version:      0.1

## Note
TODO: add back other variant fields (like `variant_allele`)